# Analytic integration of C-CST element

We tried to do the computational implementation directly and failed 🥲. So we need to build the local matrices for our element analytically and use that as a reference to fix the numerical approach.

The element is a 9-node quadrilateral where the 9-nodes are used to interpolate the x and y displacements, the 4 nodes on the vertices are used to interpolate the rotation field and the element in the facet interpolates the skew-symmetric part of the force-stress tensor.

It is a slight variation from the element described in [Guarín Variational methods for Bloch analysis paper](https://arxiv.org/pdf/2106.08893)


In [2]:
from sympy import (
    init_session,
    symbols,
    Matrix,
    S,
    zeros,
    integrate,
    diff,
    simplify,
    eye,
)

init_session()

IPython console for SymPy 1.12 (Python 3.12.2-64-bit) (ground types: python)

These commands were executed:
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.12/



In [3]:
E, rho, eta = symbols('E rho eta', positive=True)
nu, r, s = symbols('nu r s')

## Constitutive equations 

In Voigt's notation for plane-strain

In [4]:
factor = E*(1 - nu)/((1 + nu)*(1-2*nu))
factor = 1
C = factor*Matrix([
     [1,nu/(1 - nu),0], 
     [nu/(1 - nu),1,0], 
     [0,0,(1-2*nu)/(2*(1 - nu))]])

display(C)
display(C.subs(nu, 0.29).subs(E, 2))

⎡         ν           ⎤
⎢  1    ─────     0   ⎥
⎢       1 - ν         ⎥
⎢                     ⎥
⎢  ν                  ⎥
⎢─────    1       0   ⎥
⎢1 - ν                ⎥
⎢                     ⎥
⎢              1 - 2⋅ν⎥
⎢  0      0    ───────⎥
⎣              2 - 2⋅ν⎦

⎡        1          0.408450704225352          0        ⎤
⎢                                                       ⎥
⎢0.408450704225352          1                  0        ⎥
⎢                                                       ⎥
⎣        0                  0          0.295774647887324⎦

In [5]:
D = 4*eta*eye(2)
D.subs(eta, 1e4)

⎡40000.0     0   ⎤
⎢                ⎥
⎣   0     40000.0⎦

## Element formulation

### Interpolating functions

4 nodes interpolation functions

    4-----3
    |     |
    |     |
    1-----2

In [6]:
N4 = S(1)/4*Matrix([(1 - r) * (1 - s),
                   (1 + r) * (1 - s),
                   (1 + r) * (1 + s),
                   (1 - r) * (1 + s),])
N4.subs(r, -0.7745).subs(s, -0.7745)

⎡0.7872125625⎤
⎢            ⎥
⎢0.1000374375⎥
⎢            ⎥
⎢0.0127125625⎥
⎢            ⎥
⎣0.1000374375⎦

9 nodes interpolation functions

    4--7--3
    |     |
    8  9  6
    |     |
    1--5--2

In [7]:
N9 =  Matrix(
        [S(1)/4*r*s*(r - 1)*(s - 1),
         S(1)/4*r*s*(r + 1)*(s - 1),
         S(1)/4*r*s*(r + 1)*(s + 1),
         S(1)/4*r*s*(r - 1)*(s + 1),
         S(1)/2*s*(-r**2 + 1)*(s - 1),
         S(1)/2*r*(r + 1)*(-s**2 + 1),
         S(1)/2*s*(-r**2 + 1)*(s + 1),
         S(1)/2*r*(r - 1)*(-s**2 + 1),
         (-r**2 + 1)*(-s**2 + 1)])

N9.subs(r, -1).subs(s, -1)

⎡1⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [8]:
for i in range(len(N9)):
    display(simplify(diff(N9[i], r)).subs(r, -0.7745).subs(s, -0.7745))


In [9]:
for i in range(len(N4)):
    display(simplify(diff(N4[i], r)).subs(r, -0.7745).subs(s, -0.7745))


### Interpolation matrices

In [10]:
# this one is H in the numerical code
H9 = zeros(2, 18)
for i in range(9):
    H9[0, 2*i] = N9[i]
    H9[1, 2*i + 1] = N9[i]

display(H9)

⎡                                                                             
⎢r⋅s⋅(r - 1)⋅(s - 1)                       r⋅s⋅(r + 1)⋅(s - 1)                
⎢───────────────────           0           ───────────────────           0    
⎢         4                                         4                         
⎢                                                                             
⎢                                                                             
⎢                     r⋅s⋅(r - 1)⋅(s - 1)                       r⋅s⋅(r + 1)⋅(s
⎢         0           ───────────────────           0           ──────────────
⎣                              4                                         4    

                                                                              
       r⋅s⋅(r + 1)⋅(s + 1)                       r⋅s⋅(r - 1)⋅(s + 1)          
       ───────────────────           0           ───────────────────          
                4                                  

### Derivatives interpolation matrices

In [11]:
B9 = zeros(3, 18)
for i in range(9):
    B9[0, 2*i] = diff(N9[i], r)
    B9[1, 2*i + 1] = diff(N9[i], s)
    B9[2, 2*i] = diff(N9[i], s)
    B9[2, 2*i + 1] = diff(N9[i], r)


In [12]:
Bk = zeros(2, 4)
for i in range(4):
    Bk[0, i] = diff(-N4[i], s)
    Bk[1, i] = diff(N4[i], r)

display(Bk)

⎡1   r  r   1    r   1   r   1 ⎤
⎢─ - ─  ─ + ─  - ─ - ─   ─ - ─ ⎥
⎢4   4  4   4    4   4   4   4 ⎥
⎢                              ⎥
⎢s   1  1   s   s   1     s   1⎥
⎢─ - ─  ─ - ─   ─ + ─   - ─ - ─⎥
⎣4   4  4   4   4   4     4   4⎦

## Mass matrix

In [13]:
M9_inte = H9.T * H9
M9 = integrate(integrate(rho*M9_inte, [r, -1, 1]), [s, -1, 1])
display(M9.subs(rho, 1))

⎡  16                                                                         
⎢ ───      0     -4/225    0     1/225     0     -4/225    0     8/225     0  
⎢ 225                                                                         
⎢                                                                             
⎢          16                                                                 
⎢  0      ───      0     -4/225    0     1/225     0     -4/225    0     8/225
⎢         225                                                                 
⎢                                                                             
⎢                  16                                                         
⎢-4/225    0      ───      0     -4/225    0     1/225     0     8/225     0  
⎢                 225                                                         
⎢                                                                             
⎢                          16                       

## Stiffness matrix

In [14]:
Kuu_inte = B9.T * C * B9
Kuu = simplify(integrate(integrate(Kuu_inte, [r, -1, 1]), [s, -1, 1]))

In [15]:
D

⎡4⋅η   0 ⎤
⎢        ⎥
⎣ 0   4⋅η⎦

In [16]:
Kθ_inte = Bk.T * D * Bk
Kθ = simplify(integrate(integrate(Kθ_inte, [r, -1, 1]), [s, -1, 1]))

In [17]:
B_curl = zeros(1, 18)
for i in range(9):
    B_curl[0, 2*i] = diff(-N9[i], s)
    B_curl[0, 2*i + 1] = diff(N9[i], r)

display(B_curl)

⎡                                                                             
⎢  r⋅s⋅(r - 1)   r⋅(r - 1)⋅(s - 1)  r⋅s⋅(s - 1)   s⋅(r - 1)⋅(s - 1)    r⋅s⋅(r 
⎢- ─────────── - ─────────────────  ─────────── + ─────────────────  - ───────
⎣       4                4               4                4                 4 

                                                                              
+ 1)   r⋅(r + 1)⋅(s - 1)  r⋅s⋅(s - 1)   s⋅(r + 1)⋅(s - 1)    r⋅s⋅(r + 1)   r⋅(
──── - ─────────────────  ─────────── + ─────────────────  - ─────────── - ───
               4               4                4                 4           

                                                                              
r + 1)⋅(s + 1)  r⋅s⋅(s + 1)   s⋅(r + 1)⋅(s + 1)    r⋅s⋅(r - 1)   r⋅(r - 1)⋅(s 
──────────────  ─────────── + ─────────────────  - ─────────── - ─────────────
     4               4                4                 4                4    

                                           ⎛     

In [18]:
Kus = simplify(integrate(integrate(B_curl, [r, -1, 1]), [s, -1, 1]))
display(Kus.T)

⎡1/3 ⎤
⎢    ⎥
⎢-1/3⎥
⎢    ⎥
⎢1/3 ⎥
⎢    ⎥
⎢1/3 ⎥
⎢    ⎥
⎢-1/3⎥
⎢    ⎥
⎢1/3 ⎥
⎢    ⎥
⎢-1/3⎥
⎢    ⎥
⎢-1/3⎥
⎢    ⎥
⎢4/3 ⎥
⎢    ⎥
⎢ 0  ⎥
⎢    ⎥
⎢ 0  ⎥
⎢    ⎥
⎢4/3 ⎥
⎢    ⎥
⎢-4/3⎥
⎢    ⎥
⎢ 0  ⎥
⎢    ⎥
⎢ 0  ⎥
⎢    ⎥
⎢-4/3⎥
⎢    ⎥
⎢ 0  ⎥
⎢    ⎥
⎣ 0  ⎦

In [19]:
Ksθ_inte = 2*N4
Ksθ  = simplify(integrate(integrate(Ksθ_inte, [r, -1, 1]), [s, -1, 1]))
display(Ksθ)

⎡2⎤
⎢ ⎥
⎢2⎥
⎢ ⎥
⎢2⎥
⎢ ⎥
⎣2⎦